# First task

In [1]:
def convert_to_ints(string):                
    return map(lambda x: -1 if x == '' else int(x), string)

In [2]:
def get_id_lists_from_train_file(filename):
    train_file = open(filename, "rU")    
    viewed = []
    bought  = []
    list_viewed = []
    list_bought = []
    
    for line in train_file:
        splitted_line = line.split(';')   
        curr_view = convert_to_ints(splitted_line[0].split(','))
        curr_buy  = convert_to_ints(splitted_line[1].strip('\n').split(','))
        
        viewed.append(curr_view)     
        bought.append(curr_buy)
        
        list_viewed.extend(curr_view)        
        list_bought.extend(curr_buy)
    
    return viewed, bought, list_viewed, list_bought

In [3]:
tr_viewed, tr_bought, tr_list_viewed, tr_list_bought = get_id_lists_from_train_file('coursera_sessions_train.txt')

In [4]:
from collections import Counter

In [5]:
frqd_viewed = dict(Counter(tr_list_viewed))
frqd_bought = dict(Counter(tr_list_bought))

# Second task

In [258]:
def cmp_by_views(a, b):
    global frqd_viewed
    
    if a[1] in frqd_viewed and b[1] in frqd_viewed:
        if frqd_viewed[a[1]] != frqd_viewed[b[1]]:
            return frqd_viewed[b[1]] - frqd_viewed[a[1]]
        else:
            return a[0] - b[0]
    elif a[1] in frqd_viewed:
        return -1
    elif b[1] in frqd_viewed:
        return 1
    else:
        return a[0] - b[0]

In [259]:
def cmp_by_buys(a, b):
    global frqd_bought
    
    if a[1] in frqd_bought and b[1] in frqd_bought:
        if frqd_bought[a[1]] != frqd_bought[b[1]]:
            return frqd_bought[b[1]] - frqd_bought[a[1]] 
        else:
            return a[0] - b[0]
    elif a[1] in frqd_bought:
        return -1
    elif b[1] in frqd_bought:
        return 1
    else:
        return a[0] - b[0]

In [260]:
def without_repetions(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [261]:
def sort_session(session, k_first):    
    sorted_by_views = sorted(enumerate(without_repetions(session)), cmp=cmp_by_views)
    sorted_by_views = map(lambda x: x[1], sorted_by_views)
    
    sorted_by_buys  = sorted(enumerate(without_repetions(session)), cmp=cmp_by_buys)
    sorted_by_buys  = map(lambda x: x[1], sorted_by_buys)
    
    return sorted_by_views[:k_first], sorted_by_buys[:k_first]

# Third task

In [275]:
def precision_recall_k(session_viewed, session_bought, k):
    
    sorted_by_views_session, sorted_by_buys_session = sort_session(session_viewed, k)
    
    precision_by_views = 0
    precision_by_buys  = 0
    
    for bought in session_bought:                
        if bought in sorted_by_views_session:
            precision_by_views += 1
        if bought in sorted_by_buys_session:
            precision_by_buys  += 1         
    
    return float(precision_by_views) / k, float(precision_by_buys) / k, float(precision_by_views) / len(session_bought), float(precision_by_buys) / len(session_bought)

In [276]:
def mean(sequence):
    return float(sum(sequence)) / len(sequence)

In [303]:
def average_precision_recall_k(viewed, bought, k):
    
    bought_session_count = 0
    precisions_by_view = []
    precisions_by_buys = []
    recalls_by_view = []
    recalls_by_buys = []
    
    for view, buy in zip(viewed, bought):        
        if buy == [-1]: continue
        
        prec_by_view, prec_by_buy, rec_by_view, rec_by_buy = precision_recall_k(view, buy, k)
        precisions_by_view.append(prec_by_view)
        precisions_by_buys.append(prec_by_buy)
        recalls_by_view.append(rec_by_view)
        recalls_by_buys.append(rec_by_buy)
    
    return [mean(precisions_by_view), mean(precisions_by_buys), mean(recalls_by_view), mean(recalls_by_buys)]

In [322]:
avg_prc_by_views_1, avg_prc_by_buys_1, avg_rec_by_views_1, avg_rec_by_buys_1 = average_precision_recall_k(tr_viewed, tr_bought, 1)
avg_prc_by_views_5, avg_prc_by_buys_5, avg_rec_by_views_5, avg_rec_by_buys_5 = average_precision_recall_k(tr_viewed, tr_bought, 5)
print "**** TRAIN ****"
print "**VIEWS**"
print "recall_1 %.2f" % (avg_rec_by_views_1), " precision_1 %.2f" % (avg_prc_by_views_1)
print "recall_5 %.2f" % (avg_rec_by_views_5), " precision_5 %.2f" % (avg_prc_by_views_5)
print "**BUYS**"
print "recall_1 %.2f" % (avg_rec_by_buys_1), " precision_1 %.2f" % (avg_prc_by_buys_1)
print "recall_5 %.2f" % (avg_rec_by_buys_5), " precision_5 %.2f" % (avg_prc_by_buys_5)

**** TEST ****
**VIEWS**
recall_1 0.44  precision_1 0.51
recall_5 0.82  precision_5 0.21
**BUYS**
recall_1 0.69  precision_1 0.80
recall_5 0.93  precision_5 0.25


Now let's calculate for test

In [318]:
tst_viewed, tst_bought, _, _ = get_id_lists_from_train_file('coursera_sessions_test.txt')

In [323]:
avg_prc_by_views_1, avg_prc_by_buys_1, avg_rec_by_views_1, avg_rec_by_buys_1 = average_precision_recall_k(tst_viewed, tst_bought, 1)
avg_prc_by_views_5, avg_prc_by_buys_5, avg_rec_by_views_5, avg_rec_by_buys_5 = average_precision_recall_k(tst_viewed, tst_bought, 5)
print "**** TEST ****"
print "**VIEWS**"
print "recall_1 %.2f" % (avg_rec_by_views_1), " precision_1 %.2f" % (avg_prc_by_views_1)
print "recall_5 %.2f" % (avg_rec_by_views_5), " precision_5 %.2f" % (avg_prc_by_views_5)
print "**BUYS**"
print "recall_1 %.2f" % (avg_rec_by_buys_1), " precision_1 %.2f" % (avg_prc_by_buys_1)
print "recall_5 %.2f" % (avg_rec_by_buys_5), " precision_5 %.2f" % (avg_prc_by_buys_5)

**** TEST ****
**VIEWS**
recall_1 0.42  precision_1 0.48
recall_5 0.80  precision_5 0.20
**BUYS**
recall_1 0.46  precision_1 0.53
recall_5 0.82  precision_5 0.21
